#  Data Cleaning of Housing.com 
___

In [281]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [282]:
df = pd.read_csv(r'data_housing_com.csv')

In [283]:
df.head()

,location,beds1,beds2,price,area
0,"THE EAGLES, Jayanagar, Kuvempunagar, Mysore",2 BHK Apartment,1010 sq.ft built up area,[6400000],"{'value': 1010, 'unit': 'sq.ft'}"
1,"THE EAGLES, Jayanagar, Kuvempunagar, Mysore",3 BHK Apartment,1505 sq.ft built up area,[9200000],"{'value': 1505, 'unit': 'sq.ft'}"
2,"THE EAGLES, Yadavagiri, Mysore",3 BHK Apartment,1650 sq.ft built up area,[11000000],"{'value': 1650, 'unit': 'sq.ft'}"
3,"THE EAGLES, Somanatha Nagar, Mysore",3 BHK Independent House,2000 sq.ft built up area,[10500000],"{'value': 2000, 'unit': 'sq.ft'}"
4,"THE EAGLES, Sardar Vallabh Bhai Patel Nagara, ...",2 BHK Independent House,1200 sq.ft built up area,[8800000],"{'value': 1200, 'unit': 'sq.ft'}"


## Cleaning Location Data

In [284]:
df['location'] = [x[:-8] for x in df['location']]

for i,x in enumerate(df['location']):
    
    if len(x.split(','))> 1:
        df['location'][i] = x.split(',')[-1]
        
for i,x in enumerate(df['location']):
    
    if x == 'Dattagalli Third Stage':
        df['location'][i] = 'Dattagalli'
    
df.head()    

,location,beds1,beds2,price,area
0,Kuvempunagar,2 BHK Apartment,1010 sq.ft built up area,[6400000],"{'value': 1010, 'unit': 'sq.ft'}"
1,Kuvempunagar,3 BHK Apartment,1505 sq.ft built up area,[9200000],"{'value': 1505, 'unit': 'sq.ft'}"
2,Yadavagiri,3 BHK Apartment,1650 sq.ft built up area,[11000000],"{'value': 1650, 'unit': 'sq.ft'}"
3,Somanatha Nagar,3 BHK Independent House,2000 sq.ft built up area,[10500000],"{'value': 2000, 'unit': 'sq.ft'}"
4,Sardar Vallabh Bhai Patel Nagara,2 BHK Independent House,1200 sq.ft built up area,[8800000],"{'value': 1200, 'unit': 'sq.ft'}"


## Beds Column Cleaning

* A new column **Type** is added to the dataframe which consits of the type of the property

In [285]:
df['beds1'] = [x.split(' B')[0] for x in df['beds1']]

In [286]:
df['Type'] = ''
for i,x in enumerate(df['beds2']):
    if 'villa' in x.lower():
        df['Type'][i] = 'Villa'
    elif 'apartment' in x.lower():
        df['Type'][i] = 'Apartment'
    else:
        df['Type'][i] = 'House'
        
    if 'bhk' in x.lower():
        df['beds1'][i] = x.split(' B')[0].split(',')[-1]
    

In [287]:
alphabets = df[df['beds1'].str.contains('[A-Za-z]')].index
df.drop(alphabets,0,inplace=True)
df.drop(['beds2'],1, inplace=True)

In [288]:
df.rename(columns={'location':'Location','beds1':'Beds','price':'Price','area':'Area'},inplace=True)

In [289]:
df['Price'] = [x[1:-1] for x in df['Price']]

In [291]:
df['Price'] = [x.split(',')[-1] for x in df['Price']]

In [293]:
df['Area']= [x.split(',')[0][-4: ] for x in df['Area']]
df['Area'] = [int(x) for x in df['Area']]

In [297]:
df.head(20)

,Location,Beds,Price,Area,Type
0,Kuvempunagar,2,6400000,1010,House
1,Kuvempunagar,3,9200000,1505,House
2,Yadavagiri,3,11000000,1650,House
3,Somanatha Nagar,3,10500000,2000,House
4,Sardar Vallabh Bhai Patel Nagara,2,8800000,1200,House
5,Bogadi,2,5490000,1100,House
6,Bogadi,3,11500000,2130,House
7,Vijayanagar,3,6000000,1430,House
8,Nanjangud,4,77775000,5000,Apartment
9,Basavanahalli,3,6150000,643,Apartment


## Creating a Temporary DataFrame to fill out the missing values 

The missing values can be filled with the median house value in that locality

In [216]:
temp_df = df[df['Price'] != '']

In [217]:
temp_df['Beds'] = temp_df['Beds'].apply(lambda x: round(float(x)))

In [218]:
temp_df['Price'] = temp_df['Price'].apply(lambda x: int(x.split(',')[-1]))

In [219]:
temp_df.describe()

,Beds,Price,Area
count,563.000000,5.630000e+02,563.000000
mean,3.000000,1.042977e+07,1646.930728
std,1.368169,9.485249e+06,1152.253335
min,1.000000,5.000000e+05,0.000000
25%,2.000000,5.600000e+06,1000.000000
50%,3.000000,7.500000e+06,1200.000000
75%,3.000000,1.200000e+07,2000.000000
max,10.000000,1.100000e+08,8167.000000


In [220]:
temp_df['Location'] = [x.strip(' ') for x in temp_df['Location']]

### The missing datas are replaced by the median of the price for the corresponding locality

In [221]:
missing_data = temp_df.groupby(['Location']).median()
missing_data.head()

,Beds,Price,Area
Location,,,
,3.0,18000000.0,2450.0
Alanahalli Village,2.0,6450000.0,855.0
Ankanahalli,5.0,8996580.0,1147.0
Avverahalli,4.0,11640050.0,2469.5
BEML Nagar,3.0,11250000.0,1784.0


In [222]:
missing_prices = missing_data['Price']
missing_prices.head()

Location
                      18000000.0
Alanahalli Village     6450000.0
Ankanahalli            8996580.0
Avverahalli           11640050.0
BEML Nagar            11250000.0
Name: Price, dtype: float64

In [250]:
missing_df = df[df['Price'] == '']

In [235]:
missing_df.drop(23,0,inplace=True)

### Passing the missing Values to the Main Data Frame

In [298]:
for i,x in enumerate(missing_df['Location']):
    #print(x)
    indexs = missing_df.index
    try:
        df['Price'][indexs[i]] = missing_prices[x.strip(' ')]
        #print('1')
    except KeyError:
       # print('0')
        pass
      

In [277]:
missing_df.index

Int64Index([ 11,  13,  14,  19,  21,  22,  23,  24,  25,  26,  27,  28,  29,
             45,  49,  60,  61,  62,  63,  64,  65, 552, 556, 559, 560, 572,
            576, 577, 580, 583, 586, 589, 591, 594, 595, 596, 601, 604, 606],
           dtype='int64')

### DataFrame with filled values

In [300]:
df.head(20)

,Location,Beds,Price,Area,Type
0,Kuvempunagar,2,6400000,1010,House
1,Kuvempunagar,3,9200000,1505,House
2,Yadavagiri,3,11000000,1650,House
3,Somanatha Nagar,3,10500000,2000,House
4,Sardar Vallabh Bhai Patel Nagara,2,8800000,1200,House
5,Bogadi,2,5490000,1100,House
6,Bogadi,3,11500000,2130,House
7,Vijayanagar,3,6000000,1430,House
8,Nanjangud,4,77775000,5000,Apartment
9,Basavanahalli,3,6150000,643,Apartment


In [306]:
df.isnull().sum()

Location    0
Beds        0
Price       0
Area        0
Type        0
dtype: int64

In [319]:
df['Price'] = [int(x) for x in df['Price']]

In [320]:
df['Area'] = [int(x) for x in df['Area']]

In [321]:
df['Beds'] = [round(float(x)) for x in df['Beds']]

### Lets replace the missing values with a rather large number, its treated as an outlier

In [327]:
df.replace(0,-999999,inplace= True)

In [333]:
df.to_csv('data_cleaned.csv',index=False)